# find the top 10 brands from frequency counts

In [1]:
import pandas as pd

In [19]:
df = pd.read_csv("7500_comments_edmunds.csv", sep="\t") 
df = df[['Date', 'user_id', 'comments']]
df = df.dropna()

In [20]:
df.head()

,Date,user_id,comments
0,2014-02-20 14:06:00,robr2,"[someone, better, call, marketing, then:]"
1,2014-02-20 14:14:00,andys120,"[all, what?, this, whole, renaming, scheme, ha..."
2,2014-02-20 14:34:00,flightnurse,"[lol..., oh, well....]"
3,2014-02-20 15:04:00,Michaell,"[isn't, there, going, to, be, a, 4-series, ""gr..."
4,2014-02-20 15:13:00,kyfdx,[l]


In [21]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from itertools import chain
#nltk.download('punkt')
#nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tsblo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tsblo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [22]:
%%time
# tokenization
df['tokens'] = df['comments'].map(nltk.word_tokenize)
df['tokens'] = df['tokens'].map(lambda x: set(x))

# lemmatization
lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df ['tokens'].map(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['lemmatized'] = df['lemmatized'].map(lambda x: set(x))

# remove stop words
stop_words = set(stopwords.words('english'))
df['processed'] = df['lemmatized'].map(lambda x: [word for word in x if word.lower() not in stop_words])

# remove punctuations
punc = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~..."
df['processed'] = df['processed'].map(lambda x: [word for word in x if word.lower() not in punc])

# remove some other stuff and return lower case
others = ["''", "``", "n't", "l", "oh", "lol", "'m", "'s"]
df['processed'] = df['processed'].map(lambda x: [word.lower() for word in x if word.lower() not in others])

Wall time: 8.96 s


In [24]:
df.head()

,Date,user_id,comments,tokens,lemmatized,processed
0,2014-02-20 14:06:00,robr2,Someone better call marketing then:,"{marketing, :, better, call, Someone, then}","{marketing, :, better, call, Someone, then}","[marketing, better, call, someone]"
1,2014-02-20 14:14:00,andys120,All what? This whole renaming scheme has turne...,"{I, with, All, renaming, turned, that, 's, Thi...","{I, with, name, renaming, All, turned, that, '...","[name, renaming, turned, guess, hardly, collos..."
2,2014-02-20 14:34:00,flightnurse,LOL... Oh well....,"{LOL, ..., Oh, well, .}","{LOL, ..., Oh, well, .}",[well]
3,2014-02-20 15:04:00,Michaell,"Isn't there going to be a 4-series ""grand coup...","{name, that, just, Is, for, ?, 4-series, like,...","{name, that, there, Is, for, ?, 4-series, like...","[name, 4-series, like, well, 6-series, going, ..."
4,2014-02-20 15:13:00,kyfdx,l,{l},{l},[]


# Replace frequently occurring car models with brands 

In [26]:
models = pd.read_csv("models (5).csv")
models.columns = ['brand', 'model']

In [27]:
models.head()

,brand,model
0,acura,integra
1,acura,legend
2,acura,vigor
3,acura,rlx
4,acura,ilx


In [28]:
## clean brand list
models.loc[(models['brand'] == 'mercedes'),'brand'] = 'mercedes-benz'
models.loc[(models['brand'] == 'kia.'),'brand'] = 'kia'

In [29]:
def model_to_brand(word):
    if word.lower() in models['model'].values:
        brand = models[models['model'] == word].iloc[-1,0]
        print('replace ' + word + ' to ' + brand)
        return(brand)
    else:
        return(word)

In [30]:
df['processed'] = df['processed'].map(lambda x: [model_to_brand(word) for word in x])

replace 4-series to bmw
replace 6-series to bmw
replace accent to hyundai
replace 3-series to bmw
replace focus to ford
replace legacy to subaru
replace mercedes to mercedes-benz
replace focus to ford
replace camry to toyota
replace cx-7 to mazda
replace colorado to chevrolet
replace quattro to audi
replace cavalier to chevrolet
replace cruiser to chrysler
replace comfort to toyota
replace a4 to audi
replace lacrosse to buick
replace passat to volkwagen
replace ilx to acura
replace a4 to audi
replace a3 to audi
replace quattro to audi
replace a4 to audi
replace a3 to audi
replace quattro to audi
replace jetta to volkswagen
replace passat to volkwagen
replace quattro to audi
replace tsx to acura
replace a3 to audi
replace tsx to acura
replace a3 to audi
replace quattro to audi
replace a4 to audi
replace ilx to acura
replace ilx to acura
replace rlx to acura
replace tsx to acura
replace 328i to bmw
replace tsx to acura
replace tsx to acura
replace tsx to acura
replace legend to honda
rep

replace mercedes to mercedes-benz
replace city to honda
replace pilot to honda
replace cx-7 to mazda
replace focus to ford
replace pilot to honda
replace stellar to hyundai
replace quattro to audi
replace 328i to bmw
replace city to honda
replace a4 to audi
replace chevy to chevrolet
replace impala to chevrolet
replace altima to nissan
replace a4 to audi
replace city to honda
replace a3 to audi
replace a3 to audi
replace a4 to audi
replace quattro to audi
replace a6 to audi
replace genesis to hyundai
replace a3 to audi
replace mercedes to mercedes-benz
replace vw to volkswagen
replace a4 to audi
replace ilx to acura
replace civic to honda
replace ilx to acura
replace civic to honda
replace q7 to audi
replace ilx to acura
replace civic to honda
replace rsx to acura
replace q5 to audi
replace q7 to audi
replace vw to volkswagen
replace pony to hyundai
replace rsx to acura
replace civic to honda
replace vw to volkswagen
replace q5 to audi
replace q7 to audi
replace pony to hyundai
replace

replace tsx to acura
replace legacy to subaru
replace 3er to bmw
replace legacy to subaru
replace mercedes to mercedes-benz
replace quattro to audi
replace q5 to audi
replace a3 to audi
replace q3 to audi
replace 3er to bmw
replace a4 to audi
replace tsx to acura
replace tiguan to volkswagen
replace q7 to audi
replace a8 to audi
replace tsx to acura
replace pilot to honda
replace bmws to bmw
replace m3 to bmw
replace gti to volkswagen
replace m5 to bmw
replace bmws to bmw
replace bmws to bmw
replace a4 to audi
replace 328i to bmw
replace xdrive to bmw
replace 528i to bmw
replace ilx to acura
replace a4 to audi
replace accords to honda
replace accord to honda
replace fusion to ford
replace accord to honda
replace a3 to audi
replace pony to hyundai
replace ilx to acura
replace golf to volkswagen
replace civic to honda
replace a6 to audi
replace a8 to audi
replace a4 to audi
replace mustang to ford
replace golf to volkswagen
replace 370z to nissan
replace focus to ford
replace a3 to audi


replace mkz to lincoln
replace a4 to audi
replace quattro to audi
replace a3 to audi
replace ilx to acura
replace fusion to ford
replace c-class to mercedes-benz
replace mkz to lincoln
replace ilx to acura
replace a4 to audi
replace ilx to acura
replace civic to honda
replace ilx to acura
replace civic to honda
replace a4 to audi
replace fusion to ford
replace mkz to lincoln
replace focus to ford
replace ilx to acura
replace civic to honda
replace ilx to acura
replace golf to volkswagen
replace civic to honda
replace a3 to audi
replace ilx to acura
replace ilx to acura
replace ilx to acura
replace integra to acura
replace focus to ford
replace venture to toyota
replace mercedes to mercedes-benz
replace a3 to audi
replace q50 to infiniti
replace a4 to audi
replace a6 to audi
replace ilx to acura
replace malibus to chevrolet
replace versa to nissan
replace m3 to bmw
replace a3 to audi
replace jetta to volkswagen
replace ilx to acura
replace a4 to audi
replace ilx to acura
replace s60 to 

replace quattro to audi
replace quattro to audi
replace a6 to audi
replace genesis to hyundai
replace genesis to hyundai
replace outback to subaru
replace city to honda
replace 3-series to bmw
replace city to honda
replace c-class to mercedes-benz
replace benz to mercedes-benz
replace accent to hyundai
replace mercedes to mercedes-benz
replace a7 to audi
replace 3-series to bmw
replace c-class to mercedes-benz
replace benz to mercedes-benz
replace accent to hyundai
replace mercedes to mercedes-benz
replace 4-series to bmw
replace 3-series to bmw
replace s-class to mercedes-benz
replace mdx to acura
replace a6 to audi
replace genesis to hyundai
replace q7 to audi
replace malibu to chevrolet
replace fx35 to infiniti
replace a7 to audi
replace corvette to chevrolet
replace vw to volkswagen
replace ilx to acura
replace ilx to acura
replace cars to car
replace gti to volkswagen
replace fusion to ford
replace vw to volkswagen
replace vw to volkswagen
replace chevy to chevrolet
replace vw to 

...

replace leaf to nissan
replace mercedes to mercedes-benz
replace buicks to buick
replace regal to buick
replace camry to toyota
replace accord to honda
replace genesis to hyundai
replace genesis to hyundai
replace accord to honda
replace camry to toyota
replace accord to honda
replace altima to nissan
replace a4 to audi
replace camry to toyota
replace accord to honda
replace a4 to audi
replace m5 to bmw
replace a6 to audi
replace genesis to hyundai
replace bmws to bmw
replace equus to hyundai
replace a4 to audi
replace 328i to bmw
replace g25 to infiniti
replace 335i to bmw
replace fusions to ford
replace accord to honda
replace altima to nissan
replace camry to toyota
replace altima to nissan
replace m6 to bmw
replace g25 to infiniti
replace camry to toyota
replace altima to nissan
replace mazda6 to mazda
replace tsx to acura
replace altima to nissan
replace g25 to infiniti
replace mercedes to mercedes-benz
replace xts to cadillac
replace xts to cadillac
replace m5 to bmw
replace lega

replace 328i to bmw
replace city to honda
replace a4 to audi
replace jetta to volkswagen
replace yaris to toyota
replace m6 to bmw
replace 335d to bmw
replace 335i to bmw
replace 328i to bmw
replace vw to volkswagen
replace vw to volkswagen
replace 335i to bmw
replace prius to toyota
replace swift to suzuki
replace comfort to toyota
replace city to honda
replace m3 to bmw
replace 335d to bmw
replace 328i to bmw
replace vw to volkswagen
replace wrx to subaru
replace vw to volkswagen
replace 335d to bmw
replace vw to volkswagen
replace sonata to hyundai
replace optima to kia
replace vw to volkswagen
replace city to honda
replace mercedes to mercedes-benz
replace corvette to chevrolet
replace 328i to bmw
replace q50 to infiniti
replace vw to volkswagen
replace wave to pontiac
replace golf to volkswagen
replace vw to volkswagen
replace fusion to ford
replace vw to volkswagen
replace 335i to bmw
replace 3-series to bmw
replace 5-series to bmw
replace tsx to acura
replace tsx to acura
replac

replace a6 to audi
replace quattro to audi
replace a4 to audi
replace mdx to acura
replace a6 to audi
replace avalon to toyota
replace xts to cadillac
replace insight to honda
replace s60 to volvo
replace a4 to audi
replace mercedes to mercedes-benz
replace a4 to audi
replace cruiser to chrysler
replace a6 to audi
replace avalon to toyota
replace camry to toyota
replace xts to cadillac
replace mercedes to mercedes-benz
replace s80 to volvo
replace rlx to acura
replace camry to toyota
replace legend to honda
replace lacrosse to buick
replace camry to toyota
replace xts to cadillac
replace accords to honda
replace a6 to audi
replace avalon to toyota
replace camry to toyota
replace mercedes to mercedes-benz
replace pony to hyundai
replace a6 to audi
replace avalon to toyota
replace mks to lincoln
replace comfort to toyota
replace camry to toyota
replace xts to cadillac
replace mercedes to mercedes-benz
replace camry to toyota
replace avalon to toyota
replace xts to cadillac
replace a6 to 

replace 335i to bmw
replace a3 to audi
replace quattro to audi
replace aztek to pontiac
replace murano to nissan
replace pilot to honda
replace camry to toyota
replace mercedes to mercedes-benz
replace vw to volkswagen
replace passat to volkwagen
replace gti to volkswagen
replace g37 to infiniti
replace gti to volkswagen
replace vw to volkswagen
replace a3 to audi
replace accord to honda
replace seats to seat
replace accord to honda
replace quattro to audi
replace xc60 to volvo
replace a6 to audi
replace q5 to audi
replace fx35 to infiniti
replace q50 to infiniti
replace a4 to audi
replace quattro to audi
replace a6 to audi
replace q50 to infiniti
replace a6 to audi
replace q50 to infiniti
replace a4 to audi
replace q50 to infiniti
replace leaf to nissan
replace impala to chevrolet
replace chevy to chevrolet
replace city to honda
replace maxima to nissan
replace cx-9 to mazda
replace tsx to acura
replace s60 to volvo
replace mazda6 to mazda
replace comfort to toyota
replace s60 to volv

replace mercedes to mercedes-benz
replace 335i to bmw
replace a4 to audi
replace bmws to bmw
replace vw to volkswagen
replace charger to dodge
replace benz to mercedes-benz
replace mercedes to mercedes-benz
replace xts to cadillac
replace xts to cadillac
replace benz to mercedes-benz
replace ilx to acura
replace a4 to audi
replace spark to chevrolet
replace express to chevrolet
replace m6 to bmw
replace fusion to ford
replace accord to honda
replace optima to kia
replace a4 to audi
replace fusion to ford
replace accord to honda
replace optima to kia
replace pony to hyundai
replace vw to volkswagen
replace venture to toyota
replace m3 to bmw
replace benz to mercedes-benz
replace mustang to ford
replace accord to honda
replace optima to kia
replace mustang to ford
replace mustang to ford
replace a6 to audi
replace 335i to bmw
replace c-class to mercedes-benz
replace volt to chevrolet
replace mercedes to mercedes-benz
replace vw to volkswagen
replace genesis to hyundai
replace prelude to 

In [31]:
all_words = pd.Series(chain(*list(df['processed'])))
word_frequency = all_words.value_counts()
word_frequency

car                3430
bmw                2551
wa                 2300
like               2075
said               2054
one                2040
would              1965
audi               1911
ha                 1733
get                1655
new                1508
think              1482
year               1348
even               1273
time               1188
good               1185
acura              1134
much               1132
well               1125
know               1070
really             1069
drive              1023
honda              1000
still               983
doe                 983
could               978
price               973
make                973
go                  925
also                922
                   ... 
yday/today            1
preanouncing          1
2,100-2,150           1
8:30                  1
24,100                1
blindside             1
5k—almost             1
ln                    1
experiences           1
18:43.                1
greetings       

In [32]:
brand_list = models['brand'].drop_duplicates().values
no_use = ['car', 'sedan', 'seat', 'problem']
brand_list = [ x for x in brand_list if x not in no_use]

In [33]:
tmp = pd.DataFrame(word_frequency)
tmp['word'] = word_frequency.index
mask = tmp['word'].isin(brand_list)
brand_frequency = word_frequency[mask]
brand_frequency

bmw              2551
audi             1911
acura            1134
honda            1000
infiniti          662
toyota            488
volkswagen        483
mercedes-benz     411
hyundai           396
ford              376
cadillac          356
volvo             324
chevrolet         248
nissan            228
mazda             195
subaru            187
kia               179
buick             176
lincoln           135
dodge             124
chrysler          100
volkwagen          56
pontiac            38
mercury            10
mitsubishi          4
saturn              3
suzuki              3
dtype: int64

# Task A: calculate lift ratios for associations between the brands